*Click the Play button below, wait for green tick and upload the X-ray image in the deployed app.*

In [ ]:
# imports
import torch
import torchvision
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from tqdm import tqdm
from torchvision import transforms, datasets

# from google.colab import files
# print("UPLOAD MODEL : ")
# uploaded = files.upload()

!pip install anvil-uplink
!pip install --upgrade --no-cache-dir gdown

import anvil.server
anvil.server.connect("H66XJPDKGWVA4QKWQALN3KQO-AGUAQDNI26FFVN62")

id = "1I4XBcOZha5jhqopEPY9JWIioFGVHG37C"
# https://drive.google.com/file/d/1I4XBcOZha5jhqopEPY9JWIioFGVHG37C/view?usp=sharing
# file_download_link = "https://docs.google.com/uc?export=download&id=" + id 

# !wget -O pneumonia.zip "$file_download_link"

!gdown "1I4XBcOZha5jhqopEPY9JWIioFGVHG37C&confirm=t"

import anvil.media
from PIL import Image

@anvil.server.callable
def predict(file):
  with anvil.media.TempFile(file) as fileName:
    img = Image.open(fileName)
    img = img.convert('RGB')

  transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = torchvision.models.densenet121(pretrained=True).to(device)

  features = model.classifier.in_features
  model.classifier = torch.nn.Linear(features, 2).to(device)

  model.load_state_dict(torch.load('/content/pneumonia.pt'))   

  img = transform(img).to(device)

  out = model(img.unsqueeze(0))
  pred = torch.max(out, 1)[1]

  if pred == 0:
    return "Pneumonia not Detected."
  elif pred == 1:
    return "Penumonia Detected"



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1I4XBcOZha5jhqopEPY9JWIioFGVHG37C&confirm=t
To: /content/pneumonia.pt
100% 28.4M/28.4M [00:00<00:00, 153MB/s] 
